<a href="https://colab.research.google.com/github/Guimol/TBL-Brill/blob/main/TBL_GUILHERME.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Projeto de mestrado
### Algoritmo TBL-Brill

#Carregando o ambiente

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install conllu
!pip install pyconll

import conllu, pyconll, io

from conllu import parse_incr
from nltk import BrillTagger, BrillTaggerTrainer

##Neste diretório estão os recursos que utilizei para formar o bl_anotada_snl.conllu
##Está compartilhado com você e o Prof Evandro

In [ ]:
path = '/content/drive/MyDrive/MESTRADO/DISSERTAÇÃO MESTRADO/CODIGO/base_lines_consti_ud/'

In [ ]:
conll_ = path + 'bl_anotada_snl.conllu'
bl = pyconll.load_from_file(conll_)
# bl de baseline

In [ ]:
#Eu baixei o arquivo e dei upload manualmente
conll_local = '/content/bl_anotada_snl.conllu'
bl = pyconll.load_from_file(conll_local)

In [ ]:
#Acredito que era algo assim que voce queria que fosse o algoritmo para encontro de snl e palavras
#O erro aparentemente eh uma bobeira feito pelo proprio python, onde quando voce quer armazenar uma lista e em seguida limpa-la aonde ela foi armazenada tambem eh limpado automaticamente.

snl = list()
lista_palavras = list()
lista_snl = list()
lista_frases = list()

for sentence in bl:
  for word in sentence:
    # if word.xpos == 'B' or 'I':
    snl.append((word.lemma, word.deprel))
    
    lista_palavras.append((word.lemma, word.deprel))

  lista_snl.append(list(snl))
  lista_frases.append(list(lista_palavras))

  snl.clear()
  lista_palavras.clear()

In [ ]:
print(lista_snl)

[[('esse', 'nsubj'), ('ser', 'cop'), ('o', 'det'), ('avaliação', 'root'), (None, None), ('de', 'case'), ('o', 'det'), ('Instituto', 'nmod'), ('Internacional', 'flat:name'), ('de', 'case'), ('Estudos', 'nmod'), ('Estratégicos', 'flat:name'), ('de', 'case'), ('Londres', 'nmod'), ('(', 'punct'), ('IIEE', 'appos'), (')', 'punct'), (',', 'punct'), ('um', 'appos'), (None, None), ('de', 'case'), ('o', 'det'), ('mais', 'advmod'), ('importante', 'amod'), ('centro', 'nmod'), ('de', 'case'), ('pesquisa', 'nmod'), ('militar', 'amod'), (None, None), ('de', 'case'), ('o', 'det'), ('Ocidente', 'nmod'), ('.', 'punct')], [('o', 'det'), ('acordo', 'nsubj'), (None, None), ('de', 'case'), ('o', 'det'), ('dívida', 'nmod'), ('implicar', 'root'), ('o', 'det'), ('troca', 'obj'), (None, None), ('de', 'case'), ('o', 'det'), ('papel', 'nmod'), ('atual', 'amod'), ('por', 'case'), ('um', 'det'), ('combinação', 'nmod'), ('de', 'case'), ('cinco', 'nummod'), ('tipo', 'nmod'), ('diferente', 'amod'), ('de', 'case'), ('

#Brill Tagger

In [ ]:
frases_treino = lista_frases[:int(0.7 * len(lista_frases))]
frases_teste = lista_frases[int(0.7 * len(lista_frases)):]

In [ ]:
import nltk
!pip install tag_util
from tag_util import backoff_tagger

In [ ]:
t0 = nltk.DefaultTagger('DT')
t1 = nltk.UnigramTagger(frases_treino, backoff=t0)
t2 = nltk.BigramTagger(frases_treino, backoff=t1)
t3 = nltk.TrigramTagger(frases_treino, backoff=t2)

backoff_tagger = [t0,t1,t2,t3]

In [ ]:
#Links de ref
#https://www.nltk.org/book/ch05.html
#https://www.nltk.org/api/nltk.tag.sequential.html?highlight=unigramtagger#nltk.tag.sequential.UnigramTagger
#https://www.nltk.org/api/nltk.tag.brill.html?highlight=brill#module-nltk.tag.brill
#https://www.nltk.org/api/nltk.tag.brill_trainer.html?highlight=brill#module-nltk.tag.brill_trainer
#https://www.geeksforgeeks.org/nlp-brill-tagger/

In [ ]:
# Loading Libraries
from nltk.tag import brill, brill_trainer

def train_brill_tagger(initial_tagger, train_sents, **kwargs):
	templates = [
			brill.Template(brill.Pos([-1])),
			brill.Template(brill.Pos([1])),
			brill.Template(brill.Pos([-2])),
			brill.Template(brill.Pos([2])),
			brill.Template(brill.Pos([-2, -1])),
			brill.Template(brill.Pos([1, 2])),
			brill.Template(brill.Pos([-3, -2, -1])),
			brill.Template(brill.Pos([1, 2, 3])),
			brill.Template(brill.Pos([-1]), brill.Pos([1])),
			brill.Template(brill.Word([-1])),
			brill.Template(brill.Word([1])),
			brill.Template(brill.Word([-2])),
			brill.Template(brill.Word([2])),
			brill.Template(brill.Word([-2, -1])),
			brill.Template(brill.Word([1, 2])),
			brill.Template(brill.Word([-3, -2, -1])),
			brill.Template(brill.Word([1, 2, 3])),
			brill.Template(brill.Word([-1]), brill.Word([1])),
			]
	
	# Using BrillTaggerTrainer to train
	trainer = brill_trainer.BrillTaggerTrainer(
			initial_tagger, templates, deterministic = True)
	
	return trainer.train(train_sents, **kwargs)


In [ ]:
from nltk.tag import brill, brill_trainer
from nltk.tag import DefaultTagger
# from nltk.corpus import treebank
# from tag_util import train_brill_tagger

# Initializing
default_tag = DefaultTagger('NN')

# initializing training and testing set	
# train_data = treebank.tagged_sents()[:3000]
# test_data = treebank.tagged_sents()[3000:]




train_brill_tagger(default_tag,frases_treino)



initial_tag = backoff_tagger(
		frases_treino, [UnigramTagger, BigramTagger,
					TrigramTagger], backoff = default_tagger)
	
a = initial_tag.evaluate(test_data)
print ("Accuracy of Initial Tag : ", a)


NameError: ignored

In [ ]:
brill_tag = train_brill_tagger(initial_tag, train_data)
b = brill_tag.evaluate(test_data)

print ("Accuracy of brill_tag : ", b)


NameError: ignored